In [ ]:
!pip install transformers==4.30 bitsandbytes peft accelerate trl

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from datasets import load_dataset
from transformers import LlamaTokenizer
import os
import pandas as pd
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

from peft import PeftModel
from transformers import GenerationConfig
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer


In [ ]:
def get_new_prompts(question,answer):    
    return f"""A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
USER: Let's solve the question step by step as follows: 1. Comment: number of variables needed 2. Declare variables: x1, x2, x3... with commented variable description 3. Define solve(): 4. Write code with comments on thought process 5. Return final answer
Question: {question}
Solution:
ASSISTANT:
{answer}</s>
"""
new_prompts = []
for i, j in zip(question_set, answer_set):
    new_prompts.append(get_new_prompts(i,j))


## Fine Tuning

In [ ]:
# Setting for A100 - For 3090 
MICRO_BATCH_SIZE = 8 # change to 4 for 3090
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 5  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 2048  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 8
LORA_DROPOUT = 0.05

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    "TheBloke/vicuna-13B-1.1-HF",
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
    force_download = True
)


In [ ]:
# print(tokenizer.pad_token_id)
tokenizer = LlamaTokenizer.from_pretrained(
    "TheBloke/vicuna-13B-1.1-HF")
# tokenizer.pad_token_id = 2
#  = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
# tokenizer.pad_token_id

In [ ]:
tokenizer

In [ ]:
tokenizer.bos_token_id

In [ ]:
model = prepare_model_for_int8_training(model)


In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
# data = load_dataset("json", data_files="/content/global_therapy_list.json")


In [ ]:
# model = get_peft_model(model, config)
# A chat between a user and an assistant.

# USER: Hello!
# ASSISTANT: Hello!</s>
# USER: How are you?
# ASSISTANT: I am good.</s>

In [ ]:
backup_prompts = prompts_list

In [ ]:
def fix_shit(x):
    x = x.replace("""
solve()""", """
print(solve())""")
    return x


In [ ]:
old_prompts = [fix_shit(x) for x in old_prompts]

In [ ]:
prompts = old_prompts + new_prompts

In [ ]:
df = pd.DataFrame()
df['prompts'] = prompts
df.to_csv('all.csv')

In [ ]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=10,
        learning_rate=LEARNING_RATE,
        # fp16=True,
        # fsdp = 'full_shard auto_wrap',
        # fsdp_transformer_layer_cls_to_wrap = 'LlamaDecoderLayer',
        # lazy_preprocess = True,
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.03,
        logging_steps=1,
        output_dir="model_logs",
        save_total_limit=100,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("lora_drdo")

## Generation

In [ ]:
def gen_code(PROMPT):
    inputs = tokenizer(
        PROMPT,
        return_tensors="pt",
    )
    with torch.no_grad():
        out = model.generate(
            input_ids=inputs["input_ids"].cuda(),
            attention_mask=inputs["attention_mask"].cuda(),
            max_new_tokens=1024,
            do_sample=False,
            top_k=1,
            top_p=1,

            min_length = 512,
            temperature=1,
            use_cache=True
        )
    print(tokenizer.decode(out[0]))
    return (tokenizer.decode(out[0]))

### RL FINETUNE

In [ ]:
from trl import PPOTrainer,PPOConfig, AutoModelForCausalLMWithValueHead,AutoModelForSeq2SeqLMWithValueHead, create_reference_model, set_seed
# from ppo_trainer_diffValue import PPOTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel
import torch
import pandas as pd 
import os
from peft import LoraConfig, TaskType, get_peft_model

In [ ]:
from ValueHeadCustom import CustomAutoModelForCausalLMWithValueHead
import trl
trl.models.SUPPORTED_ARCHITECTURES = (
    AutoModelForCausalLMWithValueHead,
    AutoModelForSeq2SeqLMWithValueHead,
    CustomAutoModelForCausalLMWithValueHead
)

In [ ]:
config = PPOConfig(
    model_name="VICUNA_RL_MATH",
    learning_rate=1.41e-6,
    init_kl_coef=0.03,
    gamma=0.99,
    batch_size=64,
    mini_batch_size=2,
    optimize_cuda_cache=True,
    seed=21)

In [ ]:
import ast
from typing import List, Tuple
import difflib
import re

def compile_code(code: str) -> Tuple[bool, object]:
    local_vars = {}
    try:
        exec(code, local_vars)
        return True, local_vars.get('solve')
    except Exception as e:
        print(f"Error: {e}")
        return False, None

def count_matching_variables(init_code: List[str], gt_code: List[str]) -> int:
    return len(set(init_code).intersection(set(gt_code)))

def parse_operations(code: str) -> List[str]:
    operations = []
    for node in ast.walk(ast.parse(code)):
        if isinstance(node, ast.BinOp):
            operations.append(node.op.__class__.__name__)
    return operations

def extract_variables(code: str) -> List[str]:
    # Matches variable assignments of the form "variable = value # comment"
    return re.findall(r'(\w+)\s*=\s*.+?#\s*.+', code)


def reward(model_code: str, gt_code: str, max_reward: float = 1.0) -> float:
    # Compile the code
    success, model_solve = compile_code(model_code)
    R1 = max_reward if success else 0.0

    # Extract variable initialization
    model_init_variables = extract_variables(model_code)
    gt_variables = extract_variables(gt_code)
    num_correct = count_matching_variables(model_init_variables, gt_variables)
    R2 = max_reward * (num_correct / len(gt_variables)) if gt_variables else 0.0

    # Calculate R3
    R3 = 0.0
    if success and model_solve:
        try:
            gt_success, gt_solve = compile_code(gt_code)
            if gt_success and gt_solve:
                model_result = model_solve()
                gt_result = gt_solve()
                R3 = max_reward * (1 - abs(gt_result - model_result) / max(gt_result, model_result)) if max(gt_result, model_result) != 0 else 0.0
        except Exception as e:
            print(f"Error: {e}")

    # Calculate R4
    model_operations = parse_operations(model_code)
    gt_operations = parse_operations(gt_code)
    num_matching_ops = sum([1 for m_op, gt_op in zip(model_operations, gt_operations) if m_op == gt_op])
    R4 = max_reward * (num_matching_ops / len(gt_operations)) if gt_operations else 0.0

    return R1 + R2 + R3 + R4


In [ ]:
from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer,answer):
        self.texts = texts
        self.tokenizer = tokenizer
        self.answer = answer
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer.encode_plus(
            text,
            max_length=2048,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        answer = self.answer
        input_ids = encoded_text['input_ids'].squeeze()
        attention_mask = encoded_text['attention_mask'].squeeze()
        return {
            'text': text,
            'input_ids': input_ids,
            'attention_mask': attention_mask
            'answer': answer
        }

In [ ]:
train_text = pd.read_csv("train.csv")['prompts'][:-1000]
valid_text = pd.read_csv("train.csv")['prompts'][-1000:]
train_answers = pd.read_csv("train.csv")['answer'][:-1000]
valid_answers = pd.read_csv("train.csv")['answer'][-1000:]

train_data=TextDataset(train_text, tokenizer, train_answers)
valid_data=TextDataset(valid_text, tokenizer, valid_answers)

In [ ]:
model_ref = model
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer, dataset=train_data)


In [ ]:
num_epochs = 3
from tqdm import tqdm
from svampAccuracy_helper import getAccuracy
best_acc = 0
print("Training PPO model")
baseReward = 0
max_reward = 0
for epoch in tqdm(range(num_epochs)):
    print("Epoch: ", epoch)
    counter = 0
    for _, data_batch in tqdm(enumerate(ppo_trainer.dataloader)):
        # d1, d2, d3 = data_batch
        try:
            texts = data_batch['text']
            input_ids = data_batch['input_ids']
            attention_masks = data_batch['attention_mask']
            gt_code = data_batch['answer']
            rewards = []
            with torch.no_grad():
            response_tensor = model.generate(
                input_ids=input_ids.cuda(),
                attention_mask=attention_masks.cuda(),
                max_new_tokens=1024,
                do_sample=False,
                top_k=1,
                top_p=1,

                min_length = 512,
                temperature=1,
                use_cache=True
            )
            query_tensor = tokenizer.encode(texts, return_tensors='pt')[0]
            model_code = tokenizer.decode(response_tensor[0])
            match = re.search(r'(def solve\(\):.*?return.*?$)', model_code, re.DOTALL | re.MULTILINE)
            match = (match.group(1))
            reward = reward(match,gt_code)
            rewards.append(reward)
            print("reward: ",reward)
            print()
            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
            counter += 1
        except Exception as e:
            print(e)
            print(sentence)
            print("Error in training")
            print("\n")
            continue
    from svampAccuracy_helper import getAccuracy
    acc = getAccuracy(ppo_trainer.model, ppo_trainer.tokenizer, 'svamp.json')
    print("accuracySVAMP: ", acc)
    if(best_acc < acc):
        best_acc = acc